In [1]:
import numpy as np
import pandas as pd
from sklearn import svm, metrics, tree
import matplotlib.pyplot as plt
from IPython.display import display
from astropy.table import Table
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from sklearn.svm import SVC  

/Users/arthurbinda/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df= pd.read_csv('dadosProvaOnly.csv')

In [3]:
df.to_csv('dadosDiscretizados.csv', encoding='utf-8', index=False)

In [4]:
#Aqui eu tiro um aluno para teste
alunoEscolhido = 3356
dfAlunoSelecionado = df[df['Aluno'] == alunoEscolhido]

In [5]:
dfAlunoSelecionado

,Aluno,Lab,PP_Delete,Duracao,PP_Paste,Qntd_Linhas,Linhas_Log,Qntd_Linhas_Filtrado,Submissoes,Testes,Corretos,Incorretos,Branco,Nota
5192,3356,1021,0.210714,2.200522e+05,0.000000,2.500000,106.857143,2.071429,1.714286,3.785714,14,0,0,10.00000
5193,3356,1039,0.260833,8.588733e+04,0.002500,12.250000,328.333333,6.166667,1.250000,4.833333,12,0,0,10.00000
5194,3356,1069,0.370000,2.426500e+03,0.000000,11.500000,1055.000000,7.000000,2.000000,11.000000,2,0,0,10.00000
5195,3356,1070,0.123750,6.240000e+02,0.008750,8.375000,218.750000,5.875000,1.625000,9.375000,7,0,5,5.83333
5196,3356,1079,0.200000,1.972552e+06,0.000000,13.000000,190.000000,6.000000,1.000000,12.000000,0,1,9,0.00000
5197,3356,1080,0.095000,7.940000e+02,0.005000,14.500000,643.500000,10.500000,1.500000,4.500000,0,1,5,0.00000
5198,3356,1111,0.213333,3.973000e+03,0.006667,13.666667,969.333333,10.000000,2.000000,40.666667,2,0,1,6.66667
5199,3356,1131,0.135833,1.407583e+03,0.041667,24.500000,568.166667,19.750000,3.000000,14.083333,12,0,0,10.00000
5200,3356,1137,0.100000,2.256000e+03,0.010000,19.000000,1086.000000,17.000000,3.000000,15.000000,0,1,11,0.00000
5201,3356,1154,0.223333,2.215333e+03,0.016667,38.333333,1227.666667,34.000000,4.333333,30.333333,3,0,0,10.00000


In [6]:
print(int(dfAlunoSelecionado.iloc[0]['Aluno']))

3356


In [7]:
df.drop(df.loc[df['Aluno'] == alunoEscolhido].index, inplace=True)

In [8]:
df = df.groupby(['Aluno'],as_index=False).agg({'Duracao': 'mean', 'Qntd_Linhas': 'mean', 'Qntd_Linhas_Filtrado': 'mean'
                                             , 'Testes': 'mean', 'Submissoes': 'mean', 'PP_Paste': 'mean'
                                             , 'PP_Delete': 'mean', 'Linhas_Log': 'mean', 'Nota':'mean'})

In [9]:
for index, row in df.iterrows():
    if(row['Nota'] >= 5.0):
        df.replace(row['Nota'], '1',inplace=True)
    else:
        df.replace(row['Nota'], '0',inplace=True)

df[['Nota']] = df[['Nota']].astype(int)

In [10]:
df[df['Nota'] != 1]

,Aluno,Duracao,Qntd_Linhas,Qntd_Linhas_Filtrado,Testes,Submissoes,PP_Paste,PP_Delete,Linhas_Log,Nota
0,1085,762.062500,24.25,13.625,17.375,6.1875,0.001875,0.293125,516.750000,0
2,1168,232050.500000,2.5,2.5,1,1.25,0.0025,0.0275,42.000000,0
7,1373,137.000000,1,1,2,0,0,0,11.000000,0
11,1502,600.750000,9.25,5.875,1,3.375,0.00625,0.2025,380.750000,0
19,1900,1191.500000,1,3,0,1.5,0,0.335,711.500000,0
20,1913,1590.000000,18.5,7,0.5,1.5,0,0.245,962.000000,0
21,1985,832.250000,9.25,1,3.5,0.5,0.035,0.185,315.250000,0
22,2006,17227.404762,10.1667,4.69048,1.33333,1.11905,0,0.106429,205.904762,0
26,2139,13760.182944,10.4914,6.78348,6.96197,0.747403,0.041174,0.224108,265.605108,0
30,2149,15576.464286,3,2.14286,2.03571,2.89286,0.00678571,0.270714,146.571429,0


In [11]:
def analisaAcerto(i,j):
    if(i[0] == j[0]):
        return 1
    else:
        return -1

# Aqui será feito: Treino com todas as medias, e o teste será acumulativo das medias com um aluno desconhecido

In [12]:
SemNota = list(df.columns.values)
SemNota.remove('Nota')
SemNota.remove('Aluno')
print(SemNota)

['Duracao', 'Qntd_Linhas', 'Qntd_Linhas_Filtrado', 'Testes', 'Submissoes', 'PP_Paste', 'PP_Delete', 'Linhas_Log']


In [13]:
X = df[SemNota].values
y = df['Nota'].values

In [14]:
print(X[0])

[762.0625 24.25 13.625 17.375 6.1875 0.001875 0.293125 516.75]


In [15]:
print(y.shape,X.shape)

(459,) (459, 8)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25)

## Árvore de Decisão

In [17]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, max_depth=3, min_samples_leaf=5)
clf_gini.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [18]:
giniScore = clf_gini.score(X_test, y_test) 

In [19]:
y_pred = clf_gini.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 4, 13],
       [ 7, 91]])

In [20]:
y_pred = clf_gini.predict(X_test)
giniMCC = matthews_corrcoef(y_test, y_pred)

In [21]:
y_pred = clf_gini.predict(X_test)
giniF1 = precision_recall_fscore_support(y_test, y_pred, average='macro')

In [22]:
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100, max_depth=3, min_samples_leaf=5)
clf_entropy.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [23]:
entroScore = clf_entropy.score(X_test, y_test) 

In [24]:
y_pred = clf_entropy.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[12,  5],
       [10, 88]])

In [25]:
y_pred = clf_entropy.predict(X_test)
entroMCC = matthews_corrcoef(y_test, y_pred)

In [26]:
y_pred = clf_entropy.predict(X_test)
entroF1 = precision_recall_fscore_support(y_test, y_pred, average='macro')

## Random Forest

In [27]:
clfRF = RandomForestClassifier(max_depth=2, random_state=0)

In [28]:
clfRF.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [29]:
rfScore = clfRF.score(X_test, y_test) 

In [30]:
y_pred = clfRF.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 7, 10],
       [ 2, 96]])

In [31]:
y_pred = clfRF.predict(X_test)
rfMCC = matthews_corrcoef(y_test, y_pred)

In [32]:
y_pred = clfRF.predict(X_test)
rfF1 = precision_recall_fscore_support(y_test, y_pred, average='macro')

In [33]:
gini = [giniScore,giniMCC,giniF1]
entro = [entroScore,entroMCC,entroF1]
rf = [rfScore,rfMCC,rfF1]

# Classificação de aprovação na matéria

In [34]:
rows = {'Classificador','Acuracia','MCC','F1Score','Realizado'}
tabela = pd.DataFrame(columns=rows)
tabela = tabela.append({'Classificador':'Tree Gini','Acuracia':giniScore,'MCC':giniMCC,'F1Score':giniF1,'Realizado': 'Treino'}
                       , ignore_index=True)
tabela = tabela.append({'Classificador':'Tree Entropy','Acuracia':entroScore,'MCC':entroMCC,'F1Score':entroF1,'Realizado': 'Treino'}
                       , ignore_index=True)
tabela = tabela.append({'Classificador':'Random Forest','Acuracia':rfScore,'MCC':rfMCC,'F1Score':rfF1,'Realizado': 'Treino'}
                       , ignore_index=True)

In [35]:
rows = {'Aluno','Iteracao','Esperado','Resultado','IteracaoCorreta','Classificador'}
resultados = pd.DataFrame(columns=rows)

In [36]:
dfResultadoIterado = pd.DataFrame()
dfResultadoIterado = dfAlunoSelecionado.groupby(['Aluno'],as_index=False).agg({'Duracao': 'mean', 'Qntd_Linhas': 'mean', 'Qntd_Linhas_Filtrado': 'mean'
                                             , 'Testes': 'mean', 'Submissoes': 'mean', 'PP_Paste': 'mean'
                                             , 'PP_Delete': 'mean', 'Linhas_Log': 'mean', 'Nota':'mean'})
aprovacao = dfResultadoIterado['Nota'][0]
for index, row in dfAlunoSelecionado.iterrows():
    if(aprovacao >= 5.0):
        dfAlunoSelecionado.replace(row['Nota'], '1',inplace=True)
    else:
        dfAlunoSelecionado.replace(row['Nota'], '0',inplace=True)

dfAlunoSelecionado[['Nota']] = dfAlunoSelecionado[['Nota']].astype(int)
dfAlunoSelecionado = dfAlunoSelecionado.reset_index(drop=True)

In [37]:
dfAlunoSelecionado['Nota'].values[0:1]

array([0])

In [38]:
dfAlunoIterado = pd.DataFrame()
for index, row in dfAlunoSelecionado.iterrows():
    dfAlunoIterado = dfAlunoIterado.append(row)
    alunoDaVez = row.get_value(0,'Aluno')
    if(index == 0):
        SemAluno = dfAlunoIterado
        del SemAluno['Lab']
        del SemAluno['Corretos']
        del SemAluno['Incorretos']
        del SemAluno['Branco']
        del SemAluno['Nota']
        dfTeste = SemAluno
        SemAluno = list(dfAlunoIterado.columns.values)
        SemAluno.remove('Aluno')
        dfTeste = dfAlunoIterado[SemAluno]
    else:
        dfAlunoIterado['Aluno'] = dfAlunoIterado['Aluno'].astype(int)
        dfAlunoIterado['Duracao'] = dfAlunoIterado['Duracao'].astype(int)
        dfAlunoIterado['Qntd_Linhas'] = dfAlunoIterado['Qntd_Linhas'].astype(int)
        dfAlunoIterado['Qntd_Linhas_Filtrado'] = dfAlunoIterado['Qntd_Linhas_Filtrado'].astype(int)
        dfAlunoIterado['Testes'] = dfAlunoIterado['Testes'].astype(int)
        dfAlunoIterado['Submissoes'] = dfAlunoIterado['Submissoes'].astype(int)
        dfAlunoIterado['PP_Paste'] = dfAlunoIterado['PP_Paste'].astype(float)
        dfAlunoIterado['PP_Delete'] = dfAlunoIterado['PP_Delete'].astype(float)
        dfAlunoIterado['Linhas_Log'] = dfAlunoIterado['Linhas_Log'].astype(int)
        
        dfAlunoIterado = dfAlunoIterado.groupby(['Aluno'],as_index=False).agg({'Duracao': 'mean', 'Qntd_Linhas': 'mean', 'Qntd_Linhas_Filtrado': 'mean'
                                     , 'Testes': 'mean', 'Submissoes': 'mean', 'PP_Paste': 'mean'
                                     , 'PP_Delete': 'mean', 'Linhas_Log': 'mean'})
        SemAluno = list(dfAlunoIterado.columns.values)
        SemAluno.remove('Aluno')
        dfTeste = dfAlunoIterado[SemAluno]
    print("iteracao: ",index," aluno: ",alunoDaVez)
    X_1 = dfTeste.values
    y_1 = dfAlunoSelecionado['Nota'].values[1:2]
    X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, test_size=0.99)
    
    giniScore = clf_gini.score(X_test, y_test)

    y_pred = clf_gini.predict(X_test)
    giniCM = confusion_matrix(y_test, y_pred)

    y_pred = clf_gini.predict(X_test)
    giniMCC = matthews_corrcoef(y_test, y_pred)

    y_pred = clf_gini.predict(X_test)
    giniF1 = precision_recall_fscore_support(y_test, y_pred, average='macro')
    
    tabela = tabela.append({'Classificador':'Tree Gini','Acuracia':giniScore,'MCC':giniMCC,'F1Score':giniF1,'Realizado': index}
                       , ignore_index=True)
    
    iCorreto = analisaAcerto(y_test,y_pred)
    resultados = resultados.append({'Aluno':alunoDaVez,'Iteracao':index,'Esperado':y_test[0],'Resultado':y_pred[0],'IteracaoCorreta':iCorreto,'Classificador':'Tree Gini'}
                                   ,ignore_index=True)

    y_pred = clf_entropy.predict(X_test)
    entroCM = confusion_matrix(y_test, y_pred)

    y_pred = clf_entropy.predict(X_test)
    entroMCC = matthews_corrcoef(y_test, y_pred)

    y_pred = clf_entropy.predict(X_test)
    entroF1 = precision_recall_fscore_support(y_test, y_pred, average='macro')
    
    tabela = tabela.append({'Classificador':'Tree Entropy','Acuracia':entroScore,'MCC':entroMCC,'F1Score':entroF1,'Realizado': index}
                       , ignore_index=True)
    
    iCorreto = analisaAcerto(y_test,y_pred)
    resultados = resultados.append({'Aluno':alunoDaVez,'Iteracao':index,'Esperado':y_test[0],'Resultado':y_pred[0],'IteracaoCorreta':iCorreto,'Classificador':'Tree Entropy'}
                                   ,ignore_index=True)

    y_pred = clfRF.predict(X_test)
    rfCM = confusion_matrix(y_test, y_pred)

    rfScore = clfRF.score(X_test, y_test) 

    y_pred = clfRF.predict(X_test)
    rfMCC = matthews_corrcoef(y_test, y_pred)

    y_pred = clfRF.predict(X_test)
    rfF1 = precision_recall_fscore_support(y_test, y_pred, average='macro')
    
    tabela = tabela.append({'Classificador':'Random Forest','Acuracia':rfScore,'MCC':rfMCC,'F1Score':rfF1,'Realizado': index}
                       , ignore_index=True)
    
    iCorreto = analisaAcerto(y_test,y_pred)
    resultados = resultados.append({'Aluno':alunoDaVez,'Iteracao':index,'Esperado':y_test[0],'Resultado':y_pred[0],'IteracaoCorreta':iCorreto,'Classificador':'Random Forest'}
                                   ,ignore_index=True)

/Users/arthurbinda/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
/Users/arthurbinda/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/arthurbinda/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/arthurbinda/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0
antes do index     Duracao  Qntd_Linhas  Qntd_Linhas_Filtrado  Testes  Submissoes  PP_Paste  \
0  152969.5            7                     4     3.5           1   0.00125   

   PP_Delete  Linhas_Log  
0   0.235774         217  
1
antes do index    Duracao  Qntd_Linhas  Qntd_Linhas_Filtrado  Testes  Submissoes  PP_Paste  \
0  77697.5            9                   5.5       7         1.5  0.000625   

   PP_Delete  Linhas_Log  
0   0.302887         636  
2
antes do index    Duracao  Qntd_Linhas  Qntd_Linhas_Filtrado  Testes  Submissoes  PP_Paste  \
0  39160.5          8.5                     5       8           1  0.004688   

   PP_Delete  Linhas_Log  
0   0.213318         427  
3
antes do index    Duracao  Qntd_Linhas  Qntd_Linhas_Filtrado  Testes  Submissoes  PP_Paste  \
0  1005856         10.5                   5.5      10           1  0.002344   

   PP_Delete  Linhas_Log  
0   0.206659       308.5  
4
antes do index    Duracao  Qntd_Linhas  Qntd_Linhas_Filtrado  Testes  Submis

In [39]:
tabela

,Classificador,MCC,Acuracia,Realizado,F1Score
0,Tree Gini,0.197748,0.826087,Treino,"(0.6193181818181819, 0.5819327731092437, 0.593..."
1,Tree Entropy,0.544889,0.869565,Treino,"(0.7458455522971652, 0.801920768307323, 0.7684..."
2,Random Forest,0.517173,0.895652,Treino,"(0.8417190775681342, 0.6956782713085234, 0.739..."
3,Tree Gini,0.000000,1.000000,0,"(1.0, 1.0, 1.0, None)"
4,Tree Entropy,0.000000,0.869565,0,"(1.0, 1.0, 1.0, None)"
5,Random Forest,0.000000,0.000000,0,"(0.0, 0.0, 0.0, None)"
6,Tree Gini,0.000000,1.000000,1,"(1.0, 1.0, 1.0, None)"
7,Tree Entropy,0.000000,0.869565,1,"(1.0, 1.0, 1.0, None)"
8,Random Forest,0.000000,0.000000,1,"(0.0, 0.0, 0.0, None)"
9,Tree Gini,0.000000,1.000000,2,"(1.0, 1.0, 1.0, None)"


In [40]:
print(giniCM)
print(entroCM)
print(rfCM)

[[0 1]
 [0 0]]
[[1]]
[[1]]


In [41]:
resultados

,IteracaoCorreta,Classificador,Aluno,Esperado,Resultado,Iteracao
0,1,Tree Gini,3356,0,0,0
1,1,Tree Entropy,3356,0,0,0
2,-1,Random Forest,3356,0,1,0
3,1,Tree Gini,3356,0,0,1
4,1,Tree Entropy,3356,0,0,1
5,-1,Random Forest,3356,0,1,1
6,1,Tree Gini,3356,0,0,2
7,1,Tree Entropy,3356,0,0,2
8,-1,Random Forest,3356,0,1,2
9,-1,Tree Gini,3356,0,1,3
